## Title: Artificial Dataset generator
### Dataset Specification:
* The Dataset contains 1200 rows (observations) and 10 columns (variables)
* Detailed description of the variables:


| Variable Name     | Description                            | Format         | Min Value     | Max Value  | Missings | Notes                                                                                                                                                                             |
|-------------------|----------------------------------------|----------------|---------------|------------|----------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Date              | Months of observations                 | Date Type      | 1/1/2016      | 12/1/2016  | 0%       | Only the first day of each month                                                                                                                                                  |
| CompanyID         | Company's ID in the system             | Integer        | 0             | 100        | 0%       |                                                                                                                                                                                   |
| Revenue           | Monthly Company's Revenue              | Float, Decimal | 0             | 999,999.99 | 1%       | The missings are referred to the number of observations                                                                                                                           |
| Expenses          | Monthly Company's Expenses             | Float, Decimal | 0             | 500,000.00 | 1%       | The missings are referred to the number of observations                                                                                                                           |
| Profit            | Monthly Company's Profit               | Float, Decimal |               |            | 1%       | Profit = Revenue - Expenses. The missings are referred to the number of observations                                                                                              |
| LossFlag          | Monthly balance check                  | Binary         | 0             | 1          | 1%       | 0 = the company has profit, 1 = the company has negative balance. The missings are referred to the number of observations                                                         |
| Employees         | Number of employees in the company     | Integer        | 10            | 1000       | 0%       | Each company has a standard number of employees the whole year                                                                                                                    |
| Region            | Company's geographical location        | Categorical    | A, B, C, D, E |            | 0%       | The proportions of each Region are: A=25%, B=20%, C=10%, D=5%, E=40%                                                                                                              |
| BusinessValuation | Company's Market Value                 | Float, Decimal |               |            | 0%       | This value changes monthly and is equal to 3% till 10% of each company's profit                                                                                                   |
| ClosedFlag        | The company is no longer in the market | Binary         | 0             | 1          |          | 0 = No, 1 = Yes. The proportions of the "1" are: 10% of the companies than have more than 3 months negative balance and 0.5% of those with negative balance for 2 or less months. |

### Assumptions and plan to approach the challenge
1. Base on the specification and US businesses classification for companies, we can assume that we are dealing with "Small and Medium-Sized Businesses" [http://www.digium.com/blog/2016/02/18/smb-sme-large-enterprise-size-business-matters/]:
    * Employees: 0-100 is considered a small-sized business; 100-999 is considered a medium-sized business; 
    * Annual Revenue: 5-10 USD million; 
    * The 28 million small businesses in the US account for 54% of the country’s sales. 
2. According to data from the Census Bureau’s Annual Survey [https://sbecouncil.org/about-us/facts-and-data/], there were 5.6 million employer firms in the United States in 2016 with the following distribution:
    * Firms with fewer than 500 workers accounted for 99.7 percent of those businesses.
    * Firms with fewer than 100 workers accounted for 98.2 percent.
    * Firms with fewer than 20 workers made up 89.0 percent.
3. It is worth to add some extra restrictions to data set generation and make it more realistic. Here [https://www.smallbizgenius.net/by-the-numbers/small-business-statistics/] is a good simplified summary about small businesses in US, which will be taken into account to generate the data set.

#### Extra requirements for a Small Business (SMB) data set generation:
* Number of employees: 
    * 10-100: 60 % (in fact it is ~98% of SMB in US)
    * 101-500: 30%
    * 501-1000: 10%
* Profit:
    * 50% profitable (in fact it is only 40% of SMB in US)
    * 50% are not profitable monthly
    * Closing tag will be adjusted base on the requirements with addition of "consecutive months" in specification
    * Expenses will be generated randomly for the first month of the dataset. The consecutive months expenses will be generated randomly within a range of 30% from previous months average.
    *  Revenue will be generated using profit proportions and calculated expenses. Extra condition will be placed on revenue such the current month revenue can't deviate from previous month average more than 30%.
    

In [1]:
# Parameters set up

# Date
d_start = '2016-01-01'
d_end = '2017-01-01'

# Total number of businesses, ID is integer
n_busines = 100

# Revenue, expenses, profit
revenue_min = 0.
revenue_max = 999999.99
revenue_mis = 0.01
revenue_range = 0.3
expenses_min = 0.
expenses_max = 500000.
expenses_mis = 0.01
expenses_range = 0.3
profit_mis = 0.01
profit_ratio = 0.5
loss_flag_mis = 0.01


# Number of employees with proportions, integer
n_empl = {(10, 100): 0.6, (101, 500): 0.3, (501, 1000): 0.1}

# Regions, categorical. 
#The proportions of each Region are: 
regions = {'A': 0.25, 'B': 0.2, 'C': 0.1, 'D': 0.05, 'E': 0.4}

# BusinessValuation, float. Will pick up % randomly for positive profit within range.
bv_min = 0.03
bv_max = 0.1

# The company is no longer in the market, 0 = No, 1 = Yes, binary. 
# The proportions of the "1" are: 10% of the companies than have 
# more than 3 months negative balance and 0.5% of those with negative balance for 2 or less months.
nb_longer_3m = 0.1
nb_no_longer_2m = 0.05

# Set a seed to make the final results repatable
seed = 0
    
data_set_column_list = ['Date', 'CompanyID', 'Employees', 'Revenue', 'Expenses', 'Profit', 
                        'LossFlag', 'Region', 'BusinessValuation', 'ClosedFlag']

In [2]:
import pandas as pd
import numpy as np
import random
import itertools
# You need the line below only if you have problem with 'display' method in notebook
# from IPython.display import display

### Generating 'Date' dataframe for given months of observations

In [3]:
# Generate a series with the first date of the month for the given period
dates_pds = pd.date_range(d_start, d_end, freq='1M') - pd.offsets.MonthBegin(1)
display(dates_pds)

DatetimeIndex(['2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
               '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
               '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01'],
              dtype='datetime64[ns]', freq='M')

### Generating 'CompanyID' dataseries for a given number of businesses

In [4]:
# Generate companies ID and randomize array
np.random.seed(seed)
id_companies = pd.Series(np.random.permutation(n_busines), name=data_set_column_list[1])
# check the generated ids
display(id_companies.describe())

count    100.000000
mean      49.500000
std       29.011492
min        0.000000
25%       24.750000
50%       49.500000
75%       74.250000
max       99.000000
Name: CompanyID, dtype: float64

### Generate 'Employees' dataseries, with the wollowing spec for companies sizes
* company size: 10-100: 60 %; 101-500: 30%; 501-1000: 10%

In [5]:
# generating list of numbers of the companies for each category
n_empl_stratified = [int(float(n_busines)*n_empl_tmp) for n_empl_tmp in n_empl.values()] 
n_empl_all = np.empty(id_companies.size, dtype=int)
# for n_empl_category in n_empl_stratified:
np.random.seed(seed)
start_tmp = 0
for tmp_comp_cat in range(len(n_empl_stratified)):
    tmp_low_high = list(n_empl.keys())[tmp_comp_cat]
    size_tmp = n_empl_stratified[tmp_comp_cat]
    end_tmp = start_tmp + size_tmp
    n_empl_all[start_tmp:end_tmp] = np.random.randint(low=tmp_low_high[0], high=tmp_low_high[1], size=(size_tmp))
    start_tmp = end_tmp

# check number of generated employees for all companies
n_empl_all = pd.Series(n_empl_all, name=data_set_column_list[2], dtype=int)
display(n_empl_all.describe())

count    100.000000
mean     208.270000
std      244.133175
min       10.000000
25%       47.750000
50%       89.500000
75%      286.750000
max      989.000000
Name: Employees, dtype: float64

### Generate 'Expenses' dataseries randomly for the first month for each company regardless their size
* Could be changed later on base on the company size in order to make it more realistic

In [6]:
np.random.seed(seed)
expenses_first_month = np.random.uniform(low=expenses_min, high=expenses_max, size=(id_companies.size))

# check expenses
expenses_first_month = pd.Series(expenses_first_month, name=data_set_column_list[4]).round(2)
display(expenses_first_month.describe())

count       100.00000
mean     236396.92020
std      144877.00687
min        2347.74000
25%      102901.58000
50%      233740.49500
75%      342241.62750
max      494186.92000
Name: Expenses, dtype: float64

### Generate 'Profit' dataseries randomly using profit proportion 50% profitalbe and 50% non-profitable
* Could be changed later on base on the company size in order to make it more realistic

In [7]:
# First 40% of the companies will be profitable. It will be OK because the IDs are randomly shuffled already.
profit_first_month = np.empty(id_companies.size, dtype=float)
#for n_empl_category in n_empl_stratified:
random.seed(seed)
np.random.seed(seed)
for i_tmp in range(expenses_first_month.size):
    if i_tmp < profit_ratio*100:
        profit_first_month[i_tmp] = random.uniform(revenue_min+expenses_first_month[i_tmp], 
                                                       revenue_max-expenses_first_month[i_tmp])     
    else:
        profit_first_month[i_tmp] = random.uniform(-expenses_first_month[i_tmp], 0)
profit_first_month = pd.Series(profit_first_month, name=data_set_column_list[5]).round(2)
display(profit_first_month.describe())

count       100.000000
mean     216503.175600
std      326052.959524
min     -454431.650000
25%      -42720.822500
50%       89972.210000
75%      518463.032500
max      901461.830000
Name: Profit, dtype: float64

### Calculate 'Revenue' dataseries for the first month

In [8]:
# compute Revenue for the first month = Expenses + profit
revenue_first_month = pd.Series(expenses_first_month + profit_first_month, name=data_set_column_list[3])
# check revenue
display(revenue_first_month.describe())

count       100.000000
mean     452900.095800
std      370609.835043
min         828.160000
25%       90346.822500
50%      307537.850000
75%      881622.230000
max      997118.620000
Name: Revenue, dtype: float64

### Compute 'Revenue', 'Expenses', and 'Profit' for the rest of the time points. Create dataframe with the values. 
* Main condition: revenue and expenses can't deviate from previous months average more than 30% and must be within allowable range.

In [9]:
random.seed(seed)
# Combine the first month of reveniu into a dataframe with date and company IDs
final_df = pd.concat([id_companies, n_empl_all, revenue_first_month, expenses_first_month, 
                      profit_first_month], axis=1)
final_df[data_set_column_list[0]] = dates_pds[0]
# rearrange the columns
final_df = final_df[data_set_column_list[:6]]

# run computation for each company in a loop
for tmp_date in dates_pds[1:]:
    for tmp_id in id_companies[:]:
        # filter by ID
        df_tmp = final_df[final_df.CompanyID == tmp_id]
        # generate revenue
        tmp_revenue_mean = df_tmp.mean()['Revenue']
        tmp_revenue_delta = tmp_revenue_mean * 0.3
        if tmp_revenue_mean - tmp_revenue_delta < 0:
            tmp_revenue_min = revenue_min
        else:
            tmp_revenue_min = tmp_revenue_mean - tmp_revenue_delta     
        if tmp_revenue_mean + tmp_revenue_delta > revenue_max:
            tmp_revenue_max = revenue_max
        else:
            tmp_revenue_max = tmp_revenue_mean + tmp_revenue_delta
        tmp_revenue = random.uniform(tmp_revenue_min, tmp_revenue_max) 
        # generate expenses
        tmp_expenses_mean = df_tmp.mean()['Expenses']
        tmp_expenses_delta = tmp_expenses_mean * 0.3
        if tmp_expenses_mean - tmp_expenses_delta < 0:
            tmp_expenses_min = expenses_min
        else:
            tmp_expenses_min = tmp_expenses_mean - tmp_expenses_delta     
        if tmp_expenses_mean + tmp_expenses_delta > expenses_max:
            tmp_expenses_max = expenses_max
        else:
            tmp_expenses_max = tmp_expenses_mean + tmp_expenses_delta
        tmp_expenses = random.uniform(tmp_expenses_min, tmp_expenses_max) 
        # calculate profit
        tmp_profit = tmp_revenue - tmp_expenses
        # get number of employees
        tmp_n_empl_all = n_empl_all[id_companies[id_companies == tmp_id].index[0]]
        # append the results
        tmp_final_df = pd.DataFrame([[tmp_date, tmp_id, tmp_n_empl_all, round(tmp_revenue, 2), 
                                      round(tmp_expenses, 2), round(tmp_profit, 2)]], 
                                    columns=data_set_column_list[:6])
        final_df = final_df.append(tmp_final_df, ignore_index=True)    

# add loss flag
final_df[data_set_column_list[6]] = final_df.Profit <= 0

In [10]:
# Check the current dataframe
display(final_df.dtypes)
display(final_df.describe())

Date         datetime64[ns]
CompanyID             int64
Employees             int64
Revenue             float64
Expenses            float64
Profit              float64
LossFlag               bool
dtype: object

,CompanyID,Employees,Revenue,Expenses,Profit
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,49.500000,208.270000,417654.913825,223435.901017,194219.012900
std,28.878105,243.010718,332572.443062,135028.523474,295585.455455
min,0.000000,10.000000,620.690000,1709.310000,-454431.650000
25%,24.750000,47.750000,89243.627500,90379.212500,-36706.757500
50%,49.500000,89.500000,304070.275000,232387.625000,125309.745000
75%,74.250000,286.750000,739080.885000,328526.105000,465317.562500
max,99.000000,989.000000,999956.030000,499873.510000,940167.100000


### Generate 'Regions' dataseries randomly within specified proportions

In [11]:
# Add randomized Regions, categorical. 
#The proportions of each Region are: 
#regions = {'A': 0.25, 'B': 0.2, 'C': 0.1, 'D': 0.05, 'E': 0.4}
# generate a list of regions for the given number of companies and proportional distribusion
regions_list = [[tmp_region] * int(n_busines * regions[tmp_region]) for tmp_region in regions.keys()]
regions_list = list(itertools.chain(*regions_list))

# random shuffle the list
random.seed(seed)
random.shuffle(regions_list)

# create a categorical pandas series
regions_pds = pd.Series(regions_list, name=data_set_column_list[7], dtype="category")

# create pandas frame with companies ID and Regions
regions_df = pd.concat([id_companies, regions_pds], axis=1)

# merge the final_df with region df on "CompanyID"
final_df = final_df.merge(regions_df, left_on='CompanyID', right_on='CompanyID')

# Check the current dataframe
display(final_df.dtypes)
display(final_df.describe())

Date         datetime64[ns]
CompanyID             int64
Employees             int64
Revenue             float64
Expenses            float64
Profit              float64
LossFlag               bool
Region             category
dtype: object

,CompanyID,Employees,Revenue,Expenses,Profit
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,49.500000,208.270000,417654.913825,223435.901017,194219.012900
std,28.878105,243.010718,332572.443062,135028.523474,295585.455455
min,0.000000,10.000000,620.690000,1709.310000,-454431.650000
25%,24.750000,47.750000,89243.627500,90379.212500,-36706.757500
50%,49.500000,89.500000,304070.275000,232387.625000,125309.745000
75%,74.250000,286.750000,739080.885000,328526.105000,465317.562500
max,99.000000,989.000000,999956.030000,499873.510000,940167.100000


### Calculating 'BusinessValuation' randomly for profit regardless positive or negative profit withing specified profit percentage
* assume that it can be negative

In [12]:
np.random.seed(seed)
final_df[data_set_column_list[8]] = np.round(np.random.uniform(low=bv_min*final_df.Profit, 
                                            high=bv_max*final_df.Profit), decimals=2)
# Check the current dataframe
display(final_df.query('CompanyID == 26 | CompanyID == 44'))
display(final_df.dtypes)
display(final_df.describe())

,Date,CompanyID,Employees,Revenue,Expenses,Profit,LossFlag,Region,BusinessValuation
0,2016-01-01,26,54,929805.23,274406.75,655398.48,False,A,44840.36
1,2016-02-01,26,54,945682.01,316877.41,628804.60,False,A,50344.14
2,2016-03-01,26,54,809434.40,252932.49,556501.91,False,A,40175.79
3,2016-04-01,26,54,924571.51,307911.34,616660.17,False,A,42020.35
4,2016-05-01,26,54,676168.26,354345.84,321822.42,False,A,19198.59
5,2016-06-01,26,54,806109.63,366210.98,439898.65,False,A,33085.92
6,2016-07-01,26,54,978358.01,371738.10,606619.92,False,A,36780.04
7,2016-08-01,26,54,849098.38,350149.41,498948.98,False,A,46114.92
8,2016-09-01,26,54,705606.59,321700.66,383905.93,False,A,37414.09
9,2016-10-01,26,54,595487.07,305533.43,289953.65,False,A,16481.23


Date                 datetime64[ns]
CompanyID                     int64
Employees                     int64
Revenue                     float64
Expenses                    float64
Profit                      float64
LossFlag                       bool
Region                     category
BusinessValuation           float64
dtype: object

,CompanyID,Employees,Revenue,Expenses,Profit,BusinessValuation
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,49.500000,208.270000,417654.913825,223435.901017,194219.012900,12650.179725
std,28.878105,243.010718,332572.443062,135028.523474,295585.455455,20556.308295
min,0.000000,10.000000,620.690000,1709.310000,-454431.650000,-44686.900000
25%,24.750000,47.750000,89243.627500,90379.212500,-36706.757500,-2255.037500
50%,49.500000,89.500000,304070.275000,232387.625000,125309.745000,6234.630000
75%,74.250000,286.750000,739080.885000,328526.105000,465317.562500,27822.570000
max,99.000000,989.000000,999956.030000,499873.510000,940167.100000,78344.680000


### Calculating 'CloseFlag'
Conditions:
* The company is no longer in the market, 0 = No, 1 = Yes, binary. 
* The proportions of the "1" are: 10% of the companies than have more than 3 months negative balance 
* The proportions of the "1" are: 0.5% of those with negative balance for 2 or less months.

In [13]:
# compute total moths of negative balance for the given period
df_loss = final_df[['CompanyID', 'LossFlag']].groupby('CompanyID').sum()
nb_longer_3m_df = df_loss[df_loss.LossFlag > 3]
nb_no_longer_2m = df_loss[(df_loss.LossFlag > 0) & (df_loss.LossFlag <= 3)]
display(nb_no_longer_2m.describe())
display(nb_longer_3m_df.describe())

,LossFlag
count,2.000000
mean,1.500000
std,0.707107
min,1.000000
25%,1.250000
50%,1.500000
75%,1.750000
max,2.000000


,LossFlag
count,48.00000
mean,10.31250
std,2.54455
min,4.00000
25%,8.75000
50%,12.00000
75%,12.00000
max,12.00000


### Calculating 'CloseFlag', continue and modifications
#### New conditions (due to number of companies with negative balances):
* The proportions of the "1" are: 10% of the companies than have more than **3** months negative balance 
* set it to 1 in any moths after 3 negative balances occur

In [14]:
# add 'CloseFlag' column to the final_df and assing 0 to each row
final_df['CloseFlag'] = 0
final_df['CloseFlag'] = final_df['CloseFlag'].astype('bool')

# sample 10% of the companies from longer that 3 months to flag with 'CloseFlag'
close_flag_df = nb_longer_3m_df.sample(frac=nb_longer_3m, random_state=seed)
display(close_flag_df)

# find the starting month to close these companies
close_flag_list = list(close_flag_df.index)
close_date_list = []
for tmp_comp in close_flag_list:
    tmp_df = final_df[['Date', 'CompanyID', 'LossFlag']][final_df.CompanyID == tmp_comp]
    tmp_sum = 0
    tmp_date_start_end_list = []
    for tmp_date in tmp_df.Date:
        tmp_sum += int(tmp_df[tmp_df.Date == tmp_date].LossFlag)
        if tmp_sum > 3:
            tmp_date_start_end_list.append(tmp_date)
    # randomly select closing month between starting and last month of the reporting period
    random.seed(tmp_comp)
    tmp_date_start = random.choice(tmp_date_start_end_list)
    final_df.loc[(final_df.Date >= tmp_date_start) & (final_df.CompanyID == tmp_comp), 'CloseFlag'] = True   

# show one example
display(final_df[(final_df.CompanyID == tmp_comp)])

,LossFlag
CompanyID,
65,12.0
14,12.0
58,8.0
66,9.0
69,12.0


,Date,CompanyID,Employees,Revenue,Expenses,Profit,LossFlag,Region,BusinessValuation,CloseFlag
900,2016-01-01,69,485,1760.01,19593.90,-17833.89,True,E,-1547.25,False
901,2016-02-01,69,485,1942.32,25078.81,-23136.49,True,E,-1258.01,False
902,2016-03-01,69,485,1849.43,22318.22,-20468.78,True,E,-917.04,False
903,2016-04-01,69,485,1453.40,21287.78,-19834.38,True,E,-677.48,True
904,2016-05-01,69,485,1493.47,20003.17,-18509.69,True,E,-1690.34,True
905,2016-06-01,69,485,1771.62,18569.65,-16798.02,True,E,-1584.02,True
906,2016-07-01,69,485,1361.89,19031.95,-17670.06,True,E,-678.68,True
907,2016-08-01,69,485,2042.48,23789.99,-21747.51,True,E,-1161.60,True
908,2016-09-01,69,485,2116.02,25613.59,-23497.58,True,E,-993.38,True
909,2016-10-01,69,485,1398.09,21639.39,-20241.29,True,E,-771.45,True


### Check the dataframe before removing missing values

In [15]:
display(final_df.dtypes)
display(final_df.describe())

Date                 datetime64[ns]
CompanyID                     int64
Employees                     int64
Revenue                     float64
Expenses                    float64
Profit                      float64
LossFlag                       bool
Region                     category
BusinessValuation           float64
CloseFlag                      bool
dtype: object

,CompanyID,Employees,Revenue,Expenses,Profit,BusinessValuation
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,49.500000,208.270000,417654.913825,223435.901017,194219.012900,12650.179725
std,28.878105,243.010718,332572.443062,135028.523474,295585.455455,20556.308295
min,0.000000,10.000000,620.690000,1709.310000,-454431.650000,-44686.900000
25%,24.750000,47.750000,89243.627500,90379.212500,-36706.757500,-2255.037500
50%,49.500000,89.500000,304070.275000,232387.625000,125309.745000,6234.630000
75%,74.250000,286.750000,739080.885000,328526.105000,465317.562500,27822.570000
max,99.000000,989.000000,999956.030000,499873.510000,940167.100000,78344.680000


### Removing missing values according spec for 'Revenue' , 'Expenses', 'Profit', 'LossFlag'
* I assume that spec says 1% missed per month per variable

In [16]:
# create dictionary with variables and missing ratio
variable_mis = {'Revenue': revenue_mis , 'Expenses': expenses_mis, 'Profit': profit_mis, 'LossFlag': loss_flag_mis}
# loop dates and randomly pick up the compony for each variable to replace value with None
seed_start = 0
for tmp_date in dates_pds:
    for tmp_var in variable_mis.keys():
        random.seed(seed_start)
        tmp_company_id_list = random.sample(list(id_companies), int(n_busines*variable_mis[tmp_var]))
        for tmp_company_id in tmp_company_id_list:
            final_df.loc[(final_df.Date == tmp_date) & (final_df.CompanyID == tmp_company_id), tmp_var] = None
        seed_start += 20   

### Check for how many None in the dataframe per column. Each value supposed to be missed once a month

In [17]:
final_df.isna().sum()

Date                  0
CompanyID             0
Employees             0
Revenue              12
Expenses             12
Profit               12
LossFlag             12
Region                0
BusinessValuation     0
CloseFlag             0
dtype: int64

### Final step in dataframe generation is to remove values for already closed companies for the following variables:
'Revenue', 'Expenses', 'Profit', 'LossFlag', 'BusinessValuation'

In [18]:
# list of variables to work with for removal
var_final_remove = ['Revenue', 'Expenses', 'Profit', 'LossFlag', 'BusinessValuation']
# replace vaues with None
final_df.loc[final_df.CloseFlag == 1, var_final_remove] = None

# Check for None in CloseFlag==True
final_df[final_df.CloseFlag == 1]

,Date,CompanyID,Employees,Revenue,Expenses,Profit,LossFlag,Region,BusinessValuation,CloseFlag
748,2016-05-01,66,206,NaN,NaN,NaN,NaN,A,NaN,True
749,2016-06-01,66,206,NaN,NaN,NaN,NaN,A,NaN,True
750,2016-07-01,66,206,NaN,NaN,NaN,NaN,A,NaN,True
751,2016-08-01,66,206,NaN,NaN,NaN,NaN,A,NaN,True
752,2016-09-01,66,206,NaN,NaN,NaN,NaN,A,NaN,True
753,2016-10-01,66,206,NaN,NaN,NaN,NaN,A,NaN,True
754,2016-11-01,66,206,NaN,NaN,NaN,NaN,A,NaN,True
755,2016-12-01,66,206,NaN,NaN,NaN,NaN,A,NaN,True
820,2016-05-01,14,158,NaN,NaN,NaN,NaN,D,NaN,True
821,2016-06-01,14,158,NaN,NaN,NaN,NaN,D,NaN,True


## Final check of the DataFrame and saving results in to csv and xlsx

In [19]:
display(final_df.dtypes)
display(final_df.describe())
print('\nNaN STATISTICS')
display(final_df.isna().sum())
display(final_df.query('CompanyID == 26 | CompanyID == 44 | CompanyID == 58'))

Date                 datetime64[ns]
CompanyID                     int64
Employees                     int64
Revenue                     float64
Expenses                    float64
Profit                      float64
LossFlag                    float64
Region                     category
BusinessValuation           float64
CloseFlag                      bool
dtype: object

,CompanyID,Employees,Revenue,Expenses,Profit,LossFlag,BusinessValuation
count,1200.000000,1200.000000,1155.000000,1154.000000,1154.000000,1154.000000,1166.000000
mean,49.500000,208.270000,425238.138442,223853.518925,201207.182062,0.405546,13114.752487
std,28.878105,243.010718,333278.700364,134884.471611,296717.011844,0.491210,20650.726418
min,0.000000,10.000000,620.690000,1709.310000,-454431.650000,0.000000,-44686.900000
25%,24.750000,47.750000,90465.085000,90465.210000,-34573.602500,0.000000,-2132.360000
50%,49.500000,89.500000,343566.640000,231687.630000,174968.470000,0.000000,9317.785000
75%,74.250000,286.750000,744273.185000,328642.075000,475166.652500,1.000000,28616.585000
max,99.000000,989.000000,999956.030000,499873.510000,940167.100000,1.000000,78344.680000



NaN STATISTICS


Date                  0
CompanyID             0
Employees             0
Revenue              45
Expenses             46
Profit               46
LossFlag             46
Region                0
BusinessValuation    34
CloseFlag             0
dtype: int64

,Date,CompanyID,Employees,Revenue,Expenses,Profit,LossFlag,Region,BusinessValuation,CloseFlag
0,2016-01-01,26,54,929805.23,274406.75,655398.48,0.0,A,44840.36,False
1,2016-02-01,26,54,945682.01,316877.41,628804.60,0.0,A,50344.14,False
2,2016-03-01,26,54,809434.40,252932.49,556501.91,0.0,A,40175.79,False
3,2016-04-01,26,54,924571.51,307911.34,616660.17,0.0,A,42020.35,False
4,2016-05-01,26,54,676168.26,354345.84,321822.42,0.0,A,19198.59,False
5,2016-06-01,26,54,806109.63,366210.98,439898.65,0.0,A,33085.92,False
6,2016-07-01,26,54,978358.01,371738.10,606619.92,0.0,A,36780.04,False
7,2016-08-01,26,54,849098.38,350149.41,498948.98,0.0,A,46114.92,False
8,2016-09-01,26,54,705606.59,321700.66,383905.93,0.0,A,37414.09,False
9,2016-10-01,26,54,595487.07,305533.43,289953.65,0.0,A,16481.23,False


In [22]:
# Saving dataframe into .csv
final_df.to_csv('financial_artificial_dataset_AK.csv', index=False)

In [23]:
# Saving dataframe into .xlsx
# you may need to install openpyxl
#conda or pip install openpyxl
final_df.to_excel('financial_artificial_dataset_AK.xlsx', sheet_name='main', index=False)